In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.feature_selection import SelectKBest, chi2, f_classif


In [3]:
#skl.feature_selection.SelectKBest("chi2",k=10)

In [2]:
def get_data(files):
    data_matrix = np.loadtxt(files[0], dtype='i', delimiter='\t')
    data_matrix = data_matrix.T

    last_col = [0] * len(data_matrix)
    data_matrix = np.column_stack((data_matrix, last_col))

    for x in range(len(files) - 1):
        temp_matrix = np.loadtxt(files[x + 1], dtype='i', delimiter='\t')
        temp_matrix = temp_matrix.T
        last_col = [x+1] * len(temp_matrix)
        temp_matrix = np.column_stack((temp_matrix, last_col))
        data_matrix = np.concatenate((data_matrix, temp_matrix), axis=0)

    X = data_matrix[:, :-1]
    Y = data_matrix[:, -1]
    return X, Y

In [3]:
files = glob.glob('dataset/*.txt')
X, Y = get_data(files)

In [4]:
files = glob.glob('dataset/*.txt')
X, Y = get_data(files)

fvalue_selector = SelectKBest(chi2, k=10)
fvalue_selector.fit(X, Y)

rank = fvalue_selector.scores_
top_rank = []
indexes = rank.argsort()[-10:][::-1]

for index in indexes:
    top_rank.append(rank[index])
    

In [12]:
fvalue_selector.get_support(True)


array([ 2,  6, 35, 49, 51, 53, 54, 55, 56, 57], dtype=int64)